In [1]:
import pandas as pd
import requests

In [15]:
#density
response = requests.get("https://api.census.gov/data/2019/pep/population?get=NAME,DENSITY&for=state:*&key=e7b88482d553e940a4dc8d2cef76ab26d28f876a")
data = response.json()
pop_density = pd.DataFrame(data)

In [3]:
pop_density.columns = ["State","Density","number"]
pop_density = pop_density.drop(0,axis = 0)
pop_density = pop_density.drop("number",axis = 1)

In [4]:
pop_density[["Density"]] = pop_density[["Density"]].apply(pd.to_numeric)

In [5]:
#age group:under 18 and over 65
response = requests.get("https://api.census.gov/data/2019/acs/acs1/subject?get=NAME,S0101_C02_022E,S0101_C02_030E&for=state:*&key=e7b88482d553e940a4dc8d2cef76ab26d28f876a")
data = response.json()
age = pd.DataFrame(data)

In [6]:
age.columns = ["State","Under 18", "Over 65", "number"]
age = age.drop(0,axis = 0)
age[["Under 18", "Over 65"]] = age[["Under 18", "Over 65"]].apply(pd.to_numeric, errors='raise')

In [7]:
age = age.drop("number",axis = 1)

In [8]:
#sex and race
response = requests.get("https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E,B01001_002E,B01001_026E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E&for=state:*&key=e7b88482d553e940a4dc8d2cef76ab26d28f876a")
data = response.json()
sex_race = pd.DataFrame(data)


In [9]:
sex_race = sex_race.drop(0,axis = 0)
sex_race.columns = ["State","Population","male","female", "white","black","indian(Native)", "asian","hawaiian","other","number"]

In [10]:
sex_race[sex_race.columns[1:-1].to_list()] = sex_race.iloc[:,1:-1].apply(pd.to_numeric, errors = "raise")

In [11]:
for i in sex_race.iloc[:,2:-1].columns.to_list():
    sex_race[i] = round(sex_race[i]/sex_race["Population"]*100,1)

In [12]:
sex_race = sex_race.drop("number",1)

In [13]:
df1 = pd.merge(pop_density, age, how = "inner", on = "State")

In [17]:
state_demographic = pd.merge(df1,sex_race,how = "inner", on = "State")

In [18]:
state_demographic.to_csv("demographic.csv")